In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [19]:
import torch
from transformers import RobertaTokenizer, RobertaForCausalLM, Trainer, TrainingArguments
import pandas as pd
from datasets import load_dataset
from datasets import Dataset

In [4]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_code_completion.csv')
df

,instruction,input,output,prompt
0,Create a function to calculate the sum of a se...,"[1, 2, 3, 4, 5]",# Python code\ndef sum_sequence(sequence):\n ...,Below is an instruction that describes a task....
1,Generate a Python code for crawling a website ...,website: www.example.com \ndata to crawl: phon...,import requests\nimport re\n\ndef crawl_websit...,Below is an instruction that describes a task....
2,Create a Python list comprehension to get the ...,NaN,"[x*x for x in [1, 2, 3, 5, 8, 13]]",Below is an instruction that describes a task....
3,Generate a python script to perform this action.,"Given a string, remove all the consecutive dup...",def remove_duplicates(string): \n result = ...,Below is an instruction that describes a task....
4,Write a python script to generates random numb...,NaN,def generate_random_divisible_number():\n i...,Below is an instruction that describes a task....
...,...,...,...,...
18607,Generate a loop in Python which iterates throu...,NaN,for value in list_of_values:\n print(value),Below is an instruction that describes a task....
18608,Can you design a function in Python to sort a ...,"tuples = [(""b"", 0), (""a"", 1), (""c"", -1)]",def sort_tuple(tuples):\n # sort by 2nd ite...,Below is an instruction that describes a task....
18609,Generate a Python function to search an array ...,"Array = [1, 5, 10, 15, 20], Target = 15","def search(array, target):\n # Iterate over ar...",Below is an instruction that describes a task....
18610,Develop an HTTP API in Python to retrieve the ...,NaN,import requests\nfrom bs4 import BeautifulSoup...,Below is an instruction that describes a task....


### Preprocess Dataset

In [5]:
df = df[["instruction", "output"]]  # Use the necessary columns

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

# Preprocess function
def preprocess_function(examples):
    # Tokenize the prompt (input)
    inputs = tokenizer(
        examples["instruction"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # Tokenize the output (target)
    labels = tokenizer(
        examples["output"],
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    inputs["labels"] = labels
    return inputs

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

### Train the Model

In [ ]:
# Load the model
model = RobertaForCausalLM.from_pretrained("microsoft/codebert-base")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./content/drive/My Drive/Colab Notebooks/",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Optionally split this for evaluation
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()




If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-4cee80cb7ae5>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was n

Epoch,Training Loss,Validation Loss
1,1.828700,1.820438
2,1.756800,1.747688
3,1.668800,1.715990


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=6981, training_loss=1.850966223808401, metrics={'train_runtime': 2254.2646, 'train_samples_per_second': 24.769, 'train_steps_per_second': 3.097, 'total_flos': 1.4699690408595456e+16, 'train_loss': 1.850966223808401, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./code_completion_model")
tokenizer.save_pretrained("./code_completion_model")

('./code_completion_model/tokenizer_config.json',
 './code_completion_model/special_tokens_map.json',
 './code_completion_model/vocab.json',
 './code_completion_model/merges.txt',
 './code_completion_model/added_tokens.json')

In [ ]:
# Load the fine-tuned model
tokenizer = RobertaTokenizer.from_pretrained("./code_completion_model")
model = RobertaForCausalLM.from_pretrained("./code_completion_model")

### Examples

In [12]:
# Example prompt
prompt = "Create a function to calculate the sum of a sequence of numbers.\nBelow is the input:\n[1, 2, 3, 4, 5]\n"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate completion
outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

#Decode the output
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(completion)

def sum_sequence(sequence):
    sum = 0
    for num in sequence:
        sum += num
    return sum


In [13]:
# Example prompt
prompt = "Create a function to merge two sorted lists.\ndef merge_sorted_lists(list1, list2):merged_list = []while list1 and list2:if list1[0] < list2[0]:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate completion
outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

#Decode the output
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(completion)

def merge_sorted_lists(list1, list2):
    merged_list = []
    while list1 and list2:
        if list1[0] < list2[0]:
            merged_list.append(list1[0])
            list1 = list1[1:]
        else:
            merged_list.append(list2[0])
            list2 = list2[1:]
    merged_list.extend(list1)
    merged_list.extend(list2)
    return merged_list


In [14]:
# Example prompt
prompt = "Create a factorial function.\ndef factorial(n):if n == 0:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate completion
outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

#Decode the output
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(completion)

def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)


In [17]:
# Example prompt
prompt = "Create a function to sort a list of integers using the bubble sort algorithm.\ndef bubble_sort(arr):n = len(arr)for i in range(n):"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate completion
outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

# Decode the output
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(completion)

def sort_list(numbers):
    length = len(numbers)
    for i in range(length):
        for j in range(length-1-i):
            if numbers[j] < numbers[j+1]:
                numbers[j], numbers[j+1] = numbers[j+1], numbers[j]
    return numbers


In [18]:
# Example prompt
prompt = "Create a function to check if a number is prime with the provided partial code.\ndef is_prime(n):if n <= 1:return Falsefor i in range(2, n):

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate completion
outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

# Decode the output
completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(completion)

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
